# Rant: Normalización en el pipeline

## Nunca normalizar los datos como parte del pipeline de procesamiento.
- DEBE HACERSE COMO PARTE DEL MODELO

In [1]:
import torch
import torch.nn as nn
import fastai.vision.all as fv

In [2]:
def load_data(folder, img_size,batch_size):
    tmfs = fv.aug_transforms(flip_vert=False,
                            max_rotate=10,
                            max_lighting=0.25,
                            max_zoom=1.2,
                            max_warp=0.2)
    data = fv.DataBlock(blocks  = (fv.ImageBlock, fv.CategoryBlock),
                        get_items = fv.get_image_files,
                        get_y     = fv.parent_label, #Como a partir de la imagen encpontarr la categoria
                        splitter  = fv.GrandparentSplitter(), #si el abyelo es train o test
                        item_tfms = fv.Resize(img_size),#transformcaiones antes de juntar en batch y despues de juntar en batch)
                        batch_tfms = tmfs)
    return data.dataloaders(folder, bs=batch_size)#Agarra las imagenes y las regresa en batches

In [3]:
birds = load_data("/storage/birds", img_size=128, batch_size=64)

In [ ]:
learn = fv.cnn_learner(birds, fv.resnet18) # ¡GRAN ERROR MASIVO!

Así se modifican los datos. NO se pueden recuperar, se deben volver a llamar en load_data

Para evitar ese error:

In [ ]:
learn = fv.cnn_learner(birds, fv.resnet18, normalize=False )

BatchNorm se encaraga de normalizar los datos, si usamos na buena cantidad de BatchNorm

Normalizar debe ser parte del modelo (Agregar capa que normalice): Así no nos debemos volver a mmolestar por normalizar

In [ ]:
class Normalizer(nn.Module):
    def __init__(self, mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]): #De las imágenes de imagenet
        super().__init__()
        self.mean = torch.tensor(mean)[None,:,None,None] #Los pasamos a tensores
        self.std  = torch.tensor(std)[None,:,None,None]  #Ponemos las mismas dimensiones para poder broadcastear
        
    def forward(self, x):
        m = self.mean.to(x) 
        s = self.std.to(x)
        return (x - m)/s  #Regresa normalizado

In [ ]:
fv.imagenet_stats

### Para crear el modelo NORMALIZADO

In [ ]:
learn = fv.cnn_learner(birds, fv.resnet18, normalize=False)
learn.model = nn.sequential(Normalizer(), learn.model)

El modelo normaliza solo